# Build a Custom OpenAI Chatbot with ML-Driven Prompt Engineering

The code below is designed to run as-is with one exception: **the OpenAI API key must be specified**. Edit the cell below to add your API key between the double quotes.

Then, to execute each code cell, click on it and press `Shift` + `Enter` on your keyboard.

In [1]:
pip install openai==0.28

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [3]:
import openai
openai.api_key = "YOUR API KEY"

## Step 0: Inspecting Non-Customized Results

Before we perform any prompt engineering, **let's ask the OpenAI model some questions and see how it answers**.

(If you encounter an `AuthenticationError` when running this code, make sure that you have added a valid API key to the cell above and executed it.)

In [4]:
ukraine_prompt = """
Question: "When did Russia invade Ukraine?"
Answer:
"""
initial_ukraine_answer = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=ukraine_prompt,
    max_tokens=150
)["choices"][0]["text"].strip()
print(initial_ukraine_answer)

Russia invaded Ukraine in February 2014, during the Euromaidan protests and the overthrow of Ukrainian President Viktor Yanukovych. The invasion started with the seizure of the Crimean peninsula and later expanded to eastern Ukraine.


In [5]:
twitter_prompt = """
Question: "Who owns Twitter?"
Answer:
"""
initial_twitter_answer = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=twitter_prompt,
    max_tokens=150
)["choices"][0]["text"].strip()
print(initial_twitter_answer)

As of 2021, Twitter is a publicly traded company and is owned by its shareholders. The current CEO and co-founder of Twitter, Jack Dorsey, owns approximately 2% of the company's stock. Other major shareholders include institutional investors and corporations.


The model is answering this way because the training data ends in 2021. **Our task will be to provide context from 2022 to help the model answer these questions correctly.**

# Step 1: Prepare Dataset

## Loading and Wrangling Data

**The data should be loaded into a pandas `DataFrame` called `df` where each row represents a text sample, and there is only one column, `"text"`, which contains the raw text data.**

In this particular case we are collecting data from [the Wikipedia page for the year 2022](https://en.wikipedia.org/wiki/2022) and performing some data wrangling to get it into the appropriate format. Don't worry too much about the details here, since data wrangling looks different for every dataset!

In [6]:
from dateutil.parser import parse
import pandas as pd
import requests

# Get the Wikipedia page for "2022" since OpenAI's models stop in 2021
resp = requests.get("https://en.wikipedia.org/w/api.php?action=query&prop=extracts&exlimit=1&titles=2022&explaintext=1&formatversion=2&format=json")

# Load page text into a dataframe
df = pd.DataFrame()
df["text"] = resp.json()["query"]["pages"][0]["extract"].split("\n")

# Clean up text to remove empty lines and headings
df = df[(df["text"].str.len() > 0) & (~df["text"].str.startswith("=="))]

# In some cases dates are used as headings instead of being part of the
# text sample; adjust so dated text samples start with dates
prefix = ""
for (i, row) in df.iterrows():
    # If the row already has " - ", it already has the needed date prefix
    if " – " not in row["text"]:
        try:
            # If the row's text is a date, set it as the new prefix
            parse(row["text"])
            prefix = row["text"]
        except:
            # If the row's text isn't a date, add the prefix
            row["text"] = prefix + " – " + row["text"]
df = df[df["text"].str.contains(" – ")]
df

,text
0,– 2022 (MMXXII) was a common year starting on...
1,– The year saw the removal of nearly all COVI...
2,– 2022 was also dominated by wars and armed c...
9,January 1 – The Regional Comprehensive Econom...
10,January 2 – Abdalla Hamdok resigns as Prime Mi...
...,...
242,December 21–December 26 – A major winter storm...
243,December 24 – 2022 Fijian general election: Th...
244,December 29 – Brazilian football legend Pelé d...
245,December 31 – Former Pope Benedict XVI dies at...


## Generating Embeddings

We'll use the `Embedding` tooling from OpenAI [documentation here](https://platform.openai.com/docs/guides/embeddings/embeddings) to create vectors representing each row of our custom dataset.

In order to avoid a `RateLimitError` we'll send our data in batches to the `Embedding.create` function.

In [7]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 100
embeddings = []
for i in range(0, len(df), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = openai.Embedding.create(
        input=df.iloc[i:i+batch_size]["text"].tolist(),
        engine=EMBEDDING_MODEL_NAME
    )
    
    # Add embeddings to list
    embeddings.extend([data["embedding"] for data in response["data"]])

# Add embeddings list to dataframe
df["embeddings"] = embeddings
df

,text,embeddings
0,– 2022 (MMXXII) was a common year starting on...,"[9.040244185598567e-05, -0.017994549125432968,..."
1,– The year saw the removal of nearly all COVI...,"[-0.010697541758418083, -0.023004746064543724,..."
2,– 2022 was also dominated by wars and armed c...,"[-0.009615330025553703, -0.015318214893341064,..."
9,January 1 – The Regional Comprehensive Econom...,"[-0.0005404727999120951, -0.024158069863915443..."
10,January 2 – Abdalla Hamdok resigns as Prime Mi...,"[-0.015059445984661579, 0.0011320571647956967,..."
...,...,...
242,December 21–December 26 – A major winter storm...,"[-0.024877460673451424, -0.023879770189523697,..."
243,December 24 – 2022 Fijian general election: Th...,"[-0.011605652049183846, -0.009253676049411297,..."
244,December 29 – Brazilian football legend Pelé d...,"[-0.007592314388602972, 0.00409233383834362, 0..."
245,December 31 – Former Pope Benedict XVI dies at...,"[0.02339049242436886, 0.007656600326299667, -0..."


In order to avoid having to run that code again in the future, we'll save the generated embeddings as a CSV file.

In [8]:
df.to_csv("embeddings.csv")

In [9]:
! ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


If you want to stop the tutorial here and come back, you can reload `df` using this code (again adding your API key) rather than generating the embeddings again:

In [10]:
# import numpy as np
# import pandas as pd
# import openai
# openai.api_key = "YOUR API KEY"
# df = pd.read_csv("embeddings.csv", index_col=0)
# df["embeddings"] = df["embeddings"].apply(eval).apply(np.array)

In [11]:
df

,text,embeddings
0,– 2022 (MMXXII) was a common year starting on...,"[9.040244185598567e-05, -0.017994549125432968,..."
1,– The year saw the removal of nearly all COVI...,"[-0.010697541758418083, -0.023004746064543724,..."
2,– 2022 was also dominated by wars and armed c...,"[-0.009615330025553703, -0.015318214893341064,..."
9,January 1 – The Regional Comprehensive Econom...,"[-0.0005404727999120951, -0.024158069863915443..."
10,January 2 – Abdalla Hamdok resigns as Prime Mi...,"[-0.015059445984661579, 0.0011320571647956967,..."
...,...,...
242,December 21–December 26 – A major winter storm...,"[-0.024877460673451424, -0.023879770189523697,..."
243,December 24 – 2022 Fijian general election: Th...,"[-0.011605652049183846, -0.009253676049411297,..."
244,December 29 – Brazilian football legend Pelé d...,"[-0.007592314388602972, 0.00409233383834362, 0..."
245,December 31 – Former Pope Benedict XVI dies at...,"[0.02339049242436886, 0.007656600326299667, -0..."


# Step 2: Create a Function that Finds Related Pieces of Text for a Given Question

What we are implementing here is similar to a search engine or recommendation algorithm. We want to sort all of the rows of our dataset from least relevant to most relevant.

This will use the embeddings that we generated previously in order to compare the vectorized version of our question to the vectorized versions of the rows of the dataset.

In [12]:
from openai.embeddings_utils import get_embedding, distances_from_embeddings

def get_rows_sorted_by_relevance(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """
    
    # Get embeddings for the question text
    question_embeddings = get_embedding(question, engine=EMBEDDING_MODEL_NAME)
    
    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )
    
    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy


Let's test that out for a couple different questions:

In [13]:
get_rows_sorted_by_relevance("When did Russia invade Ukraine?", df)

,text,embeddings,distances
51,March 2 – Russian invasion of Ukraine: Russia ...,"[0.000819627195596695, -0.018423309549689293, ...",0.109220
79,April 3 – Russian invasion of Ukraine: As Russ...,"[-0.012132329866290092, -0.012398474849760532,...",0.111454
220,November 11 – Russian invasion of Ukraine: Ukr...,"[-0.012297285720705986, -0.013996293768286705,...",0.115655
179,September 21 – Russian invasion of Ukraine: Fo...,"[-0.025439122691750526, -0.02206767350435257, ...",0.116853
206,October 29 – Russian invasion of Ukraine: In r...,"[-0.009954243898391724, -0.030414661392569542,...",0.117845
...,...,...,...
74,March 31 – Expo 2020 closes in Dubai after a 6...,"[-0.003126838244497776, -0.04655766487121582, ...",0.292121
245,December 31 – Former Pope Benedict XVI dies at...,"[0.02339049242436886, 0.007656600326299667, -0...",0.293282
219,"November 11 – The cryptocurrency exchange FTX,...","[0.002455633133649826, -0.025356147438287735, ...",0.294034
228,November 20 – 2022 Nepalese general election: ...,"[-0.004346429370343685, -0.0007038800977170467...",0.295630


In [14]:
get_rows_sorted_by_relevance("Who owns Twitter?", df)

,text,embeddings,distances
201,October 28 – Elon Musk completes his $44 billi...,"[-0.010110262781381607, -0.01613353192806244, ...",0.172873
97,April 25 – Elon Musk reaches an agreement to a...,"[-0.011496664956212044, -0.01462513953447342, ...",0.182213
22,January 24 – The federal government under Scot...,"[-0.008650281466543674, -0.008442467078566551,...",0.248117
219,"November 11 – The cryptocurrency exchange FTX,...","[0.002455633133649826, -0.025356147438287735, ...",0.262778
230,"November 30 – OpenAI releases ChatGPT, an arti...","[-0.011260921135544777, -0.014411374926567078,...",0.264672
...,...,...,...
229,November 21 – A 5.6 earthquake strikes near Ci...,"[0.004116300027817488, -0.004122782498598099, ...",0.324566
163,August 28 – 2022 Pakistan floods: Pakistan dec...,"[-0.011932751163840294, -0.02291550487279892, ...",0.325970
170,September 5 – A 6.8 earthquake strikes Luding ...,"[0.01265291590243578, 0.01628030464053154, 0.0...",0.326603
174,September 12 – September 2022 Armenia–Azerbaij...,"[-0.012970877811312675, -0.008715740405023098,...",0.327323


# Step 3: Create a Function that Composes a Text Prompt

Building on that sorted list of rows, we're going to select the create a text prompt that provides context to a `Completion` model in order to help it answer a question. The outline of the prompt looks like this:

```
Answer the question based on the context below, and if the
question can't be answered based on the context, say "I don't
know"

Context:

{context}

---

Question: {question}
Answer:
```

We want to fit as much of our dataset as possible into the "context" part of the prompt without exceeding the number of tokens allowed by the `Completion` model, which is currently 4,000. So we'll loop over the dataset, counting the tokens as we go, and stop when we hit the limit. Then we'll join that list of text data into a single string and add it to the prompt.

In [15]:
import tiktoken

def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")
    
    # Count the number of tokens in the prompt template and question
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

{}

---

Question: {}
Answer:"""
    
    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))
    
    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:
        
        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count
        
        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)
    

Now let's test that out! We'll use a `max_token_count` below the actual limit just to keep the output shorter and more readable.

In [16]:
print(create_prompt("When did Russia invade Ukraine?", df, 200))


Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

March 2 – Russian invasion of Ukraine: Russia captures its first large city, the Black Sea port of Kherson, as shelling intensifies across many parts of Ukraine, including civilian areas.

###

April 3 – Russian invasion of Ukraine: As Russia's forces retreat from areas near Kyiv, it is accused by Ukraine of war crimes, amid mounting evidence of indiscriminate civilian killings, including the Bucha massacre.

###

November 11 – Russian invasion of Ukraine: Ukrainian forces recapture Kherson, the only regional capital to be taken by Russia since the start of the war.

---

Question: When did Russia invade Ukraine?
Answer:


In [17]:
print(create_prompt("Who owns Twitter?", df, 100))


Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

October 28 – Elon Musk completes his $44 billion acquisition of Twitter.

###

April 25 – Elon Musk reaches an agreement to acquire the social media network Twitter (which he later rebrands as X) for $44 billion USD, which later closes in October.

---

Question: Who owns Twitter?
Answer:


# Step 4: Create a Function that Answers a Question

Our final step is to send that text prompt to a `Completion` model and parse the model output!

In [18]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

def answer_question(
    question, df, max_prompt_tokens=1800, max_answer_tokens=150
):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model
    
    If the model produces an error, return an empty string
    """
    
    prompt = create_prompt(question, df, max_prompt_tokens)
    
    try:
        response = openai.Completion.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""
        

Now that we have all of the code complete, let's test it out!

In [19]:
custom_ukraine_answer = answer_question("When did Russia invade Ukraine?", df)
print(custom_ukraine_answer)

The Russian invasion of Ukraine began on February 24, 2022.


In [20]:
custom_twitter_answer = answer_question("Who owns Twitter?", df)
print(custom_twitter_answer)

I don't know


Below we compare answers with and without our custom prompt:

In [21]:
print(f"""
When did Russia invade Ukraine?

Original Answer: {initial_ukraine_answer}
Custom Answer:   {custom_ukraine_answer}

Who owns Twitter?
Original Answer: {initial_twitter_answer}
Custom Answer:   {custom_twitter_answer}
""")


When did Russia invade Ukraine?

Original Answer: Russia invaded Ukraine in February 2014, during the Euromaidan protests and the overthrow of Ukrainian President Viktor Yanukovych. The invasion started with the seizure of the Crimean peninsula and later expanded to eastern Ukraine.
Custom Answer:   The Russian invasion of Ukraine began on February 24, 2022.

Who owns Twitter?
Original Answer: As of 2021, Twitter is a publicly traded company and is owned by its shareholders. The current CEO and co-founder of Twitter, Jack Dorsey, owns approximately 2% of the company's stock. Other major shareholders include institutional investors and corporations.
Custom Answer:   I don't know



## Summary

You just used unsupervised machine learning to perform prompt engineering for custom OpenAI chat responses!

In this example, we provided context from 2022 news headlines to answer questions about current events. Try finding your own dataset for some other custom task!